# Introduction to text mining

After this lesson, students will be able to:

1. Explain the difference between a code block and a text block in Jupyter Notebooks
2. Write a short command in Python and execute the command
3. Execute pre-written Python code to create a word frequency through time plot
4. Describe how to choose words for searching
5. Update Python code to perform word frequency calculations on different newspaper title
6. Create a plot showing word frequency over time on a chosen newspaper and time frame

If you are not familiar with text data mining, take a look at this nice [StoryMap](https://storymaps.arcgis.com/stories/cd7e273c42cd4ab6b6ce3fa89c13132c) that introduces the idea of text data mining and what we can do with it.

## 1. Explain the difference between a code block and a markdown block in Jupyter Notebooks
Jupyter notebooks allow us to mix in computer programming code with readable text. Each notebook is broken into several "blocks", or sections. A block can _either_ have code (a code block) or text (a markdown block). The text you are reading right now is in a markdown block, but we will start by looking at code blocks. We can tell the difference because code blocks are shown with gray shading. We can _run_, or execute a block by clicking the gray box and pressing the &#x25B6; Run button at the top of the window.

In [ ]:
print("Welcome to Python")

We will work more with code blocks in a bit, but what about markdown blocks? Markdown blocks are where we can write text and use some simple styling. You can change the text of the markdown block by clicking on the block, pressing "Enter" and then editing the content. Try this out on this text block; when you finished adding the text, press the Run button again to have the text formatted.

We will not be doing too much work with markdown blocks, but they can be very useful if you want to write a report and include the code and results of your work.

## 2. Write a short command in Python and execute the command
Click on the code block below and type in the command `print(My name is Jeff)`, but replace "Jeff" with your name. When you finish writing the command, execute the code block by pressing the Run button.

In [ ]:
# Write a print command to print your name

You probably got a syntax error. Why? Compare your command to the first print command that we ran at the beginning of the lesson. What is different? In the code block below, re-write the command to print your name with the syntax we used at the beginning of the lesson.

In [ ]:
# Re-write the command to print your name

This exercise is meant to demonstrate that when we pass computers text information, like "My name is Jeff", we need to wrap the text in quotation marks. Missing quotation marks is a common mistake we encounter when doing text data mining in Python.

You also probably noticed the instructions in the code block start with a pound sign (`#`) at the beginning of the line. This is known as the comment character, and is how we can write human-readable notes in a code block.

## 3. Execute pre-written Python code to create a word frequency through time plot

We are going to have more opportunities to play with Python code, but let us pause for a moment to consider what we are going to be doing in terms of text data mining. For this project, we have three newspaper titles published in southern Arizona. Those newspapers were scanned and the text was digitized by the Library of Congress. (More information about the newspapers and the digitization process can be found at [https://chroniclingamerica.loc.gov/](https://chroniclingamerica.loc.gov/)).

What _we_ are going to do is look at the frequency of certain words over time. That is, we are interested to see how certain words are used in these papers over time. This allows us to identify trends over time without actually reading four years worth of newspapers.

There are many more things you can do with text data mining in Python, but they are beyond the scope of this workshop. If you are interested, Library Carpentry has an introduction to text mining lesson at [http://librarycarpentry.org/lc-tdm/index.html](http://librarycarpentry.org/lc-tdm/index.html).

But for now, let us start by looking for the frequency of the word "influenza" in the _Bisbee Daily Review_ during the years right before and during the global influenza pandemic.

We need to provide the following pieces of information:

+ The name of the folder to look in corresponding to the newspaper of interest
+ The years to consider, here 1917 and 1918,
+ The words to look for, here "influenza"
+ The language the newspaper is written in, in this case English

In [ ]:
# Store information in variables
title = "bisbee-daily-review"
year_list = ['1917', '1918']
my_words = ['influenza']
language = "english"

We can check our work by writing `print` commands on all of these variables. The first code block is done for you; fill in the remaining three.

In [ ]:
# Print value stored in title variable
print(title)

In [ ]:
# Print value stored in year_list

In [ ]:
# Print value stored in my_words

In [ ]:
# Print value stored in language

We also need to tell Python that there are additional packages to load. What does this mean? We will need to use additional programs, written in Python, to perform our text data mining. No need to change anything in the code block, but you will need to execute the block (click once on the code block and press the Run button at the top of the window). It might take a few moments for this code block to run; you will know it is done because the asterisk that appears in the square brackets to the left of the block (`In [*]`) will be replaced by a number (probably 9, `In [9]`).

In [ ]:
# Load additional packages
# for data tables
import pandas

# for file navigation
import os

# for pattern matching in filenames
import re

# for text data mining
import nltk

# for stopword corpora for a variety of languages
from nltk.corpus import stopwords

# for splitting data into individual words
from nltk.tokenize import RegexpTokenizer

# for automated text cleaning
import digcol as dc

# download the stopwords for several languages
nltk.download('stopwords')

# for drawing the plot
import plotly.express as px

The next code block does all the heavy lifting of reading in the text of each day's newspaper, calculating the relative frequency of the word influenza, and finally drawing a plot of the frequency over time. You do not need to change anything in the code block, but you should run it by clicking on the code block and pressing the Run button (or clicking on it, holding down the Control key, and pressing "Enter").

In [ ]:
################################################################################
# No need to edit anything in this code block
################################################################################

# Creating the pattern of filenames based on years to match
years = "|"
years = years.join(year_list)
pattern = "(" + years + ")([0-9]{4})*"
date_pattern = re.compile(pattern)


# Location of files with text for a day's paper
volume_path = "data/sample/" + title + "/volumes/"
my_volumes = os.listdir(volume_path)

# Use date pattern from above to restrict to dates of interest
my_volumes = list(filter(date_pattern.match, my_volumes))

# Sort them for easier bookkeeping
my_volumes.sort()

# Create a table that will hold the relative frequency for each date
dates = []
for one_file in my_volumes:
    one_date = str(one_file[0:4]) + "-" + str(one_file[4:6]) + "-" + str(one_file[6:8])
    dates.append(one_date)

# Add those dates to a data frame
results_table = pandas.DataFrame(dates, columns = ["Date"])

# Set all frequencies to zero
results_table["Frequency"] = 0.0

# Cycle over all issues and do relative frequency calculations
for issue in my_volumes:
    issue_text = dc.CleanText(filename = volume_path + issue, language = language)
    issue_text = issue_text.clean_list
    
    # Create a table with words
    word_table = pandas.Series(issue_text)

    # Calculate relative frequencies of all words in the issue
    word_freqs = word_table.value_counts(normalize = True)
    
    # Pull out only values that match words of interest
    my_freqs = word_freqs.filter(my_words)
    
    # Get the total frequency for words of interest
    total_my_freq = my_freqs.sum()
    
    # Format the date from the name of the file so we know where to put
    # the data in our table
    issue_date = str(issue[0:4]) + "-" + str(issue[4:6]) + "-" + str(issue[6:8])
    
    # Add the date & relative frequency to our data table
    results_table.loc[results_table["Date"] == issue_date, "Frequency"] = total_my_freq
    
# Analyses are all done, plot the figure
my_figure = px.line(results_table, x = "Date", y = "Frequency")
my_figure.show()

Perhaps not surprisingly, there is little use of the word "influenza" in 1917 or the first half of 1918 but it really picks up after October 1918.

It is worth mentioning what this plot is showing. On the horizontal axis is Date and the vertical axis is Frequency. What the code above does, for each day's paper, is first count the total number of words in the entire paper. It then counts how many times the word we are interested in, "influenza", occurs in that day's paper. Using those numbers, it then calculates the relative frequency of the word:

$$
\textrm{Frequency} = {{\textrm{# times influenza occurs}}\over{\textrm{total number of words in issue}}}
$$

By looking at this _relative_ frequency, it provides information about how much proportional space the newspaper is providing to the words in question and allows us to compare issues that have different lengths (i.e. a 2-page paper versus a 20-page paper).

## 4. Describe how to choose words for searching

When running text data mining analyses like these, we need to carefully consider what words we are searching for, and what the results tell us. In the case above, we searched for the word "influenza", but even back then, this was often shortened to "flu". Right now, our plot above does not capture that, because we did not tell Python to search for that word. We can use the code block below, updating the list of words to look for, and plot the data again. The only change you need to make is to add "flu" to the list stored in `my_words`. Take a look at the values of `year_list` for an example of syntax for separating items in a list.

In [ ]:
# Include "flu" in words to look for
title = "bisbee-daily-review"
year_list = ['1917', '1918'] # Each item is separated by a comma
my_words = ['influenza']     # Change this variable only, then run this code block
language = "english"

################################################################################
# No need to edit anything below here
################################################################################

# Creating the pattern of filenames based on years to match
years = "|"
years = years.join(year_list)
pattern = "(" + years + ")([0-9]{4})*"
date_pattern = re.compile(pattern)

# Location of files with text for a day's paper
volume_path = "data/sample/" + title + "/volumes/"
my_volumes = os.listdir(volume_path)

# Use date pattern from above to restrict to dates of interest
my_volumes = list(filter(date_pattern.match, my_volumes))

# Sort them for easier bookkeeping
my_volumes.sort()

# Create a table that will hold the relative frequency for each date
dates = []
for one_file in my_volumes:
    one_date = str(one_file[0:4]) + "-" + str(one_file[4:6]) + "-" + str(one_file[6:8])
    dates.append(one_date)

# Add those dates to a data frame
results_table = pandas.DataFrame(dates, columns = ["Date"])

# Set all frequencies to zero
results_table["Frequency"] = 0.0

# Cycle over all issues and do relative frequency calculations
for issue in my_volumes:
    issue_text = dc.CleanText(filename = volume_path + issue, language = language)
    issue_text = issue_text.clean_list
    
    # Create a table with words
    word_table = pandas.Series(issue_text)

    # Calculate relative frequencies of all words in the issue
    word_freqs = word_table.value_counts(normalize = True)
    
    # Pull out only values that match words of interest
    my_freqs = word_freqs.filter(my_words)
    
    # Get the total frequency for words of interest
    total_my_freq = my_freqs.sum()
    
    # Format the date from the name of the file so we know where to put
    # the data in our table
    issue_date = str(issue[0:4]) + "-" + str(issue[4:6]) + "-" + str(issue[6:8])
    
    # Add the date & relative frequency to our data table
    results_table.loc[results_table["Date"] == issue_date, "Frequency"] = total_my_freq
    
# Analyses are all done, plot the figure
my_figure = px.line(results_table, x = "Date", y = "Frequency")
my_figure.show()

Comparing this plot to the one we saw before, the pattern is pretty similar, although there do appear many more mentions of "flu" in 1917.

## 5. Update python code to perform word frequency calculations on different newspaper title

We can do a similar analysis for one of the other newspaper titles that we have data for. In this case, we want to look at the frequency of those words in the Spanish language paper, _El Tucsonense_. We will need to make three changes to the code below:

1. The title of the paper
2. The words to look for
3. The lanugage of the paper

All three of these points warrant some discussion. If you just want to head to the code, go ahead and skip the points below. The comments in the code block indicate the changes to make before you execute the code block.

1. **The title of the paper**. The paper we are going to look at is _El Tucsonense_, but we cannot just replace "bisbee-daily-review" with "El Tucsonense". This is because the data are organized in folders corresponding to, but not exactly the same as, the title of the paper. Each newspaper title has a corresponding folder named with (1) all lowercase letters and (2) dashes instead of spaces. So the data for _Bisbee Daily Review_ are in the folder "bisbee-daily-review" and the data for _El Tucsonense_ are in the folder "el-tucsonense". With this in mind, be sure to update the value for the title appropriately.
2. **The words to look for**. This highlights how important it is that we consider the results of text data mining in the context of the language being analyzed. For convenience sake, we will still search for "influenza", and replace "flu" with "gripe" and "gripa". However, the latter two, gripe and gripa, are sometimes used colloquially in Mexico to refer to the common cold, and not just the influenza virus. So while we use these two terms in place of "flu", it warrants consideration that we may be analyzing the text data of _El Tucsonense_ differently than we did for the _Bisbee Daily Review_ data.
3. **The lanugage of the paper**. Why is this step necessary? Did we not accommodate the language change by updating the search terms? Not entirely. When text data mining, one of the first steps is to remove words that are known as [stop words](https://en.wikipedia.org/wiki/Stop_words). These are words that _generally_ contribute little meaning to the piece of text and we wish to ignore in analyses. In English, these include words like "and", "I", and "the"; Spanish stop words include "y", "yo", and "la". We set the language so that the stop words of the appropriate language are removed before analysis. It would not be appropriate to just remove all stop words of every language, because there is the very real possibility that the stop words of one language may actually be words of interest in another language (consider the "yo" of Spanish compared to the colloquial term "yo" in English which have a fairly different meaning, and the latter may be something we are interested in).

In [ ]:
title = "bisbee-daily-review"   # Change the title of the paper
year_list = ['1917', '1918']
my_words = ['influenza', 'flu'] # Translate these to Spanish
language = "english"            # Update this as appropriate

title = "el-tucsonense"
year_list = ['1917', '1918']
my_words = ['influenza', "gripe", "gripa"]
language = "spanish"

################################################################################
# No need to edit anything below here
################################################################################

# Creating the pattern of filenames based on years to match
years = "|"
years = years.join(year_list)
pattern = "(" + years + ")([0-9]{4})*"
date_pattern = re.compile(pattern)

# Location of files with text for a day's paper
volume_path = "data/sample/" + title + "/volumes/"
my_volumes = os.listdir(volume_path)

# Use date pattern from above to restrict to dates of interest
my_volumes = list(filter(date_pattern.match, my_volumes))

# Sort them for easier bookkeeping
my_volumes.sort()

# Create a table that will hold the relative frequency for each date
dates = []
for one_file in my_volumes:
    one_date = str(one_file[0:4]) + "-" + str(one_file[4:6]) + "-" + str(one_file[6:8])
    dates.append(one_date)

# Add those dates to a data frame
results_table = pandas.DataFrame(dates, columns = ["Date"])

# Set all frequencies to zero
results_table["Frequency"] = 0.0

# Cycle over all issues and do relative frequency calculations
for issue in my_volumes:
    issue_text = dc.CleanText(filename = volume_path + issue, language = language)
    issue_text = issue_text.clean_list
    
    # Create a table with words
    word_table = pandas.Series(issue_text)

    # Calculate relative frequencies of all words in the issue
    word_freqs = word_table.value_counts(normalize = True)
    
    # Pull out only values that match words of interest
    my_freqs = word_freqs.filter(my_words)
    
    # Get the total frequency for words of interest
    total_my_freq = my_freqs.sum()
    
    # Format the date from the name of the file so we know where to put
    # the data in our table
    issue_date = str(issue[0:4]) + "-" + str(issue[4:6]) + "-" + str(issue[6:8])
    
    # Add the date & relative frequency to our data table
    results_table.loc[results_table["Date"] == issue_date, "Frequency"] = total_my_freq
    
# Analyses are all done, plot the figure
my_figure = px.line(results_table, x = "Date", y = "Frequency")
my_figure.show()

This plot for _El Tucsonense_ is similar to what we saw for _Bisbee Daily Review_, although there might be a slightly later uptick in the use of influenza-related terms in _El Tucsonense_.

## 6. Create a plot showing word frequency over time on a chosen newspaper and time frame

Now it is your turn. Based on what we have done before, choose a title, years of interest, and terms you would like to search for. There are three newspaper titles to choose from, and all three have papers from 1917-1919. Be sure to update the language as appropriate.

In [ ]:
title = "border-vidette"        # "bisbee-daily-review", "border-vidette", or "el-tucsonense"
year_list = ['1918', '1919']    # any or all of: 1917, 1918, 1919
my_words = ['influenza', "flu"] # search terms of interest
language = "english"            # "english" or "spanish"

################################################################################
# No need to edit anything below here
################################################################################

# Creating the pattern of filenames based on years to match
years = "|"
years = years.join(year_list)
pattern = "(" + years + ")([0-9]{4})*"
date_pattern = re.compile(pattern)

# Location of files with text for a day's paper
volume_path = "data/sample/" + title + "/volumes/"
my_volumes = os.listdir(volume_path)

# Use date pattern from above to restrict to dates of interest
my_volumes = list(filter(date_pattern.match, my_volumes))

# Sort them for easier bookkeeping
my_volumes.sort()

# Create a table that will hold the relative frequency for each date
dates = []
for one_file in my_volumes:
    one_date = str(one_file[0:4]) + "-" + str(one_file[4:6]) + "-" + str(one_file[6:8])
    dates.append(one_date)

# Add those dates to a data frame
results_table = pandas.DataFrame(dates, columns = ["Date"])

# Set all frequencies to zero
results_table["Frequency"] = 0.0

# Cycle over all issues and do relative frequency calculations
for issue in my_volumes:
    issue_text = dc.CleanText(filename = volume_path + issue, language = language)
    issue_text = issue_text.clean_list
    
    # Create a table with words
    word_table = pandas.Series(issue_text)

    # Calculate relative frequencies of all words in the issue
    word_freqs = word_table.value_counts(normalize = True)
    
    # Pull out only values that match words of interest
    my_freqs = word_freqs.filter(my_words)
    
    # Get the total frequency for words of interest
    total_my_freq = my_freqs.sum()
    
    # Format the date from the name of the file so we know where to put
    # the data in our table
    issue_date = str(issue[0:4]) + "-" + str(issue[4:6]) + "-" + str(issue[6:8])
    
    # Add the date & relative frequency to our data table
    results_table.loc[results_table["Date"] == issue_date, "Frequency"] = total_my_freq
    
# Analyses are all done, plot the figure
my_figure = px.line(results_table, x = "Date", y = "Frequency")
my_figure.show()

### Closing

If you want more practice or want to do some analyses with a larger data set, head over to the lesson at [https://mybinder.org/v2/gh/jcoliver/dig-coll-borderlands/master?filepath=Text-Mining-Template.ipynb](https://mybinder.org/v2/gh/jcoliver/dig-coll-borderlands/master?filepath=Text-Mining-Template.ipynb).

For a longer, more in-depth version of this lesson, see [https://mybinder.org/v2/gh/jcoliver/dig-coll-borderlands/master?filepath=Text-Mining-Long.ipynb](https://mybinder.org/v2/gh/jcoliver/dig-coll-borderlands/master?filepath=Text-Mining-Long.ipynb).

If you want even _more_ resources, check these out:

+ Examples of text data mining:
    + Benjamin Schmidt and Mitch Fraas. [The Language of the State of the Union](https://www.theatlantic.com/politics/archive/2015/01/the-language-of-the-state-of-the-union/384575/). _The Atlantic_ (January 18, 2015). 
    + Lincoln Mullen. [America’s Public Bible: Biblical Quotations in U.S. Newspapers](https://americaspublicbible.org/).(2016).
+ Text data mining in Python:
    + Quinn Dombrowski, Tassie Gniady, and David Kloster, "Introduction to Jupyter Notebooks," The Programming Historian 8 (2019), [https://doi.org/10.46430/phen0087](https://doi.org/10.46430/phen0087).
    + William J. Turkel and Adam Crymble, "Counting Word Frequencies with Python," The Programming Historian 1 (2012), [https://doi.org/10.46430/phen0003](https://doi.org/10.46430/phen0003).


If you have any questions or comments on this lesson, look at the [project's GitHub page](https://github.com/jcoliver/dig-coll-borderlands) and open a new issue if you don't find an answer there.